In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [7]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [9]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals =  pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]

    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 8866
200 / 8866
300 / 8866
400 / 8866
500 / 8866
600 / 8866
700 / 8866
800 / 8866
900 / 8866
1000 / 8866
1100 / 8866
1200 / 8866
1300 / 8866
1400 / 8866
1500 / 8866
1600 / 8866
1700 / 8866
1800 / 8866
1900 / 8866
2000 / 8866
2100 / 8866
2200 / 8866
2300 / 8866
2400 / 8866
2500 / 8866
2600 / 8866
2700 / 8866
2800 / 8866
2900 / 8866
3000 / 8866
3100 / 8866
3200 / 8866
3300 / 8866
3400 / 8866
3500 / 8866
3600 / 8866
3700 / 8866
3800 / 8866
3900 / 8866
4000 / 8866
4100 / 8866
4200 / 8866
4300 / 8866
4400 / 8866
4500 / 8866
4600 / 8866
4700 / 8866
4800 / 8866
4900 / 8866
5000 / 8866
5100 / 8866
5200 / 8866
5300 / 8866
5400 / 8866
5500 / 8866
5600 / 8866
5700 / 8866
5800 / 8866
5900 / 8866
6000 / 8866
6100 / 8866
6200 / 8866
6300 / 8866
6400 / 8866
6500 / 8866
6600 / 8866
6700 / 8866
6800 / 8866
6900 / 8866
7000 / 8866
7100 / 8866
7200 / 8866
7300 / 8866
7400 / 8866
7500 / 8866
7600 / 8866
7700 / 8866
7800 / 8866
7900 / 8866
8000 / 8866
8100 / 8866
8200 / 8866
8300 / 8866
8400 / 8866
8

In [10]:
games_df = pd.concat(games, ignore_index=True)
games_df.to_csv("nba_games.csv")

/Users/sohilramachandra/opt/anaconda3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:2106: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
